In [1]:
// SPDX-FileCopyrightText:  Copyright 2024 Roland Csaszar
// SPDX-License-Identifier: AGPL-3.0-or-later
//
// Project:  Umil
// File:     gap-buffer.go
// Date:     04.Feb.2024
//
// =============================================================================

import (
	"strings"
	"unicode/utf8"
)

// The index in the current line is `start - newLines.start`.
// `wantsCol` is the rune column (not byte column!) the cursor wants to hold when going up or down.
type GapBuffer struct {
	start    int
	end      int
	wantsCol int
	data     []byte
	lines    lineBuffer
}

const (
	defaultCapacity = 1024
	minGap          = 16
	lineCapFactor   = 10 // 1/10th of the default capacity, 102
	minLineCap      = 10
	growFactor      = 2
	minLineGap      = 4
)

// `start` is the index of the start of the current (the location of `GapBuffer.start`) line.
type lineBuffer struct {
	start   int
	end     int
	lengths []int
}

func newLineBuf(c int) *lineBuffer {
	cR := max(c/lineCapFactor, minLineCap)
	lb := &lineBuffer{start: 0, end: cR, lengths: make([]int, cR)}

	return lb
}

func newLineBufStr(s string, c int) *lineBuffer {
	l := newLineBuf(c)
	l.insert(s, 0)

	return l
}

func (l *lineBuffer) size() int {
	return len(l.lengths)
}

func (l *lineBuffer) lastIdx() int {
	return len(l.lengths) - 1
}

func (l *lineBuffer) up() {
	if l.start < 1 {
		return
	}
	l.end--
	l.lengths[l.end] = l.lengths[l.start]
	l.start--
}

func (l *lineBuffer) upDel() {
	if l.start < 1 {
		return
	}
	l.start--
}

func (l *lineBuffer) down() {
	if l.end > l.lastIdx() {
		return
	}
	l.start++
	l.lengths[l.start] = l.lengths[l.end]
	l.end++
}

func (l *lineBuffer) downDel() {
	if l.end > l.lastIdx() {
		return
	}
	l.end++
}

func (l *lineBuffer) del() {
	if l.lengths[l.start] == 0 {
		return
	}

	l.lengths[l.start]--
}

func newlineSplit(str string) []string {
	line, rest, _ := strings.Cut(str, "\n")
	lines := make([]string, 0)
	lines = append(lines, line)

	for rest != "" {
		line, rest, _ = strings.Cut(rest, "\n")
		lines = append(lines, line)
	}

	if rest == "" && line == "" {
		lines = append(lines, "")
	}

	return lines
}

func lineLengths(str string) []int {
	lines := newlineSplit(str)

	lens := make([]int, 0, len(lines))

	for i := range lines {
		lens = append(lens, len(lines[i])+1)
	}

	return lens
}

// grow resizes the line buffer by `growFactor` times its current size and
// copies the existing data.
func (l *lineBuffer) grow() {
	tmp := make([]int, growFactor*l.size())
	_ = copy(tmp, l.lengths[:l.start])
	nE := len(tmp) - (l.size() - l.end)
	_ = copy(tmp[nE:], l.lengths[l.end:])
	l.end = nE
	l.lengths = tmp
}

func (l *lineBuffer) insert(str string, pos int) {
	strLen := len(str)

	if strLen == 0 {
		return
	}

	lens := lineLengths(str)
	relPos := pos - l.curLineStart()
	lens[0] += relPos

	lens[len(lens)-1] += l.curLineEnd() - pos

	if l.end-l.start < len(lens)+1 {
		l.grow()
	}

	for idx := range lens {
		l.lengths[l.start+idx] = lens[idx]
	}

	l.start += len(lens) - 1
}

func (l *lineBuffer) curLine() int {
	return l.start + 1
}

func (l *lineBuffer) curLineLength() int {
	return l.lengths[l.start]
}

func (l *lineBuffer) curLineEnd() int {
	sum := 0
	for i := range l.lengths[:l.start+1] {
		sum += l.lengths[i]
	}

	return sum - 1 // returns the current last character which is not a newline.
}

func (l *lineBuffer) curLineStart() int {
	if l.start == 0 {
		return 0
	}

	sum := 0
	for i := range l.lengths[:l.start] {
		sum += l.lengths[i]
	}

	return sum
}

func (l *lineBuffer) lastLine() bool {
	return l.end == l.size()
}

func (g *GapBuffer) String() string {
	var strB strings.Builder

	cursor := "_|_"

	strB.Grow(g.start + len(g.data) - g.end + len(cursor))
	strB.Write(g.data[:g.start])
	strB.WriteString(cursor)
	strB.Write(g.data[g.end:])

	return strB.String()
}

func (g *GapBuffer) StringPair() (left string, right string) {
	return string(g.data[:g.start]), string(g.data[g.end:])
}

func NewCap(size int) *GapBuffer {
	return &GapBuffer{
		start:    0,
		end:      size,
		wantsCol: 0,
		data:     make([]byte, size),
		lines:    *newLineBuf(size),
	}
}

func New() *GapBuffer {
	return NewCap(defaultCapacity)
}

func NewStrCap(s string, c int) *GapBuffer {
	size := max(c, len(s)*growFactor)
	dat := make([]byte, size)
	sIdx := copy(dat, s)
	lines := newLineBufStr(s, size)
	runeCol := 0
	lineStart := lines.curLineStart()

	if lineStart < sIdx {
		runeCol = utf8.RuneCount(dat[lineStart:sIdx])
	}

	return &GapBuffer{
		start:    sIdx,
		end:      size,
		wantsCol: runeCol,
		data:     dat,
		lines:    *lines,
	}
}

func NewStr(s string) *GapBuffer {
	return NewStrCap(s, defaultCapacity)
}

func (g *GapBuffer) Col() int {
	if g.start < g.lines.curLineStart() {
		return 0
	}

	return g.start - g.lines.curLineStart()
}

func (g *GapBuffer) RuneCol() int {
	if g.start < g.lines.curLineStart() {
		return 0
	}

	return utf8.RuneCount(g.data[g.lines.curLineStart():g.start])
}

func (g *GapBuffer) LineLength() int {
	return g.lines.curLineLength()
}

func (g *GapBuffer) Line() int {
	return g.lines.curLine()
}

func (g *GapBuffer) LineCol() (line int, col int) {
	return g.lines.curLine(), g.Col() + 1
}

func (g *GapBuffer) LineRuneCol() (line int, runeCol int) {
	return g.lines.curLine(), g.RuneCol() + 1
}

func (g *GapBuffer) LeftDel() {
	if g.start < 1 {
		return
	}

	r, d := utf8.DecodeLastRune(g.data[:g.start])
	g.start -= d

	if r == '\n' {
		g.lines.upDel()
	} else {
		g.lines.del()
	}

	g.wantsCol = g.RuneCol()
}

func (g *GapBuffer) RightDel() {
	if g.end > len(g.data)-1 {
		return
	}

	r, d := utf8.DecodeRune(g.data[g.end:])
	g.end += d

	if r == '\n' {
		g.lines.downDel()
	} else {
		g.lines.del()
	}
}

func (g *GapBuffer) LeftMv() {
	if g.start < 1 {
		return
	}

	rChar, d := utf8.DecodeLastRune(g.data[:g.start])
	g.end -= d

	_ = copy(g.data[g.end:], g.data[g.start-d:g.start])
	g.start -= d

	if rChar == '\n' {
		g.lines.up()
	}

	g.wantsCol = g.RuneCol()
}

func (g *GapBuffer) RightMv() {
	if g.start > len(g.data)-2 {
		return
	}

	if g.end > len(g.data)-1 {
		return
	}

	r, d := utf8.DecodeRune(g.data[g.end:])
	_ = copy(g.data[g.start:], g.data[g.end:g.end+d])
	g.start += d
	g.end += d

	if r == '\n' {
		g.lines.down()
	}

	g.wantsCol = g.RuneCol()
}

func (g *GapBuffer) UpMv() {
	if g.lines.curLine() == 1 {
		return
	}

	g.lines.up()
	lineStart := g.lines.curLineStart()
	newStart := lineStart
	max := g.lines.curLineEnd()
	runeCnt := 0

	for idx := lineStart; idx < max+1; {
		newStart = idx

		if runeCnt == g.wantsCol {
			break
		}

		_, d := utf8.DecodeRune(g.data[idx:])
		idx += d
		runeCnt++
	}

	g.end -= (g.start - newStart)
	_ = copy(g.data[g.end:], g.data[newStart:g.start])
	g.start = newStart
}

func (g *GapBuffer) DownMv() {
	if g.lines.end > g.lines.lastIdx() {
		return
	}

	newLine := g.lines.curLineEnd() + 1 - g.start
	if g.lines.curLineLength() == 0 || g.lines.lastLine() {
		newLine--
	}

	idx := newLine
	runeCnt := 0

	for g.end+idx < len(g.data) && g.data[g.end+idx] != '\n' {
		if runeCnt == g.wantsCol {
			break
		}

		_, d := utf8.DecodeRune(g.data[g.end+idx:])
		if g.end+idx+d > len(g.data)-1 {
			break
		}

		idx += d
		runeCnt++
	}

	// runtime error: slice bounds out of range [1014:1013]
	_ = copy(g.data[g.start:], g.data[g.end:g.end+idx])
	g.start += idx
	g.end += idx

	if idx == 0 {
		g.start++
		g.end++
	}
	// if we are at the last line, we can move one step further to the right
	if g.end == len(g.data)-1 && runeCnt < g.wantsCol {
		g.data[g.start] = g.data[g.end]
		g.start++
		g.end++
	}

	g.lines.down()
}

// grow resizes the gap buffer by `growFactor` times its current size and copies
// the existing data.
func (g *GapBuffer) grow() {
	tmp := make([]byte, len(g.data)*growFactor)
	_ = copy(tmp, g.data[:g.start])
	nE := len(tmp) - (len(g.data) - g.end)
	_ = copy(tmp[nE:], g.data[g.end:])
	g.end = nE
	g.data = tmp
}

func (g *GapBuffer) Insert(str string) {
	if g.end-g.start < len(str)+1 {
		g.grow()
	}

	g.lines.insert(str, g.start)
	l := copy(g.data[g.start:], str)
	g.start += l
	g.wantsCol = g.RuneCol()
}


In [2]:
%%

d := NewStrCap("Hi\nYožč\n0123456",11)
d.Insert("78901234567")
d.LeftMv()
d.LeftMv()
d.LeftMv()
d.LeftMv()
d.LeftMv()
fmt.Println(d)
fmt.Printf("LEFT %+v\n",*d)
d.UpMv()
d.UpMv()
d.DownMv()
fmt.Println(d)
fmt.Printf("DOWN %+v\n",*d)
d.DownMv()
fmt.Println(d)
fmt.Printf("DOWN %+v\n",*d)
d.DownMv()
fmt.Println(d)
fmt.Printf("DOWN %+v\n",*d)
d.Insert("\n")
fmt.Println(d)
fmt.Printf("NEWL %+v\n",*d)

Hi
Yožč
0123456789012_|_34567
LEFT {start:23 end:29 wantsCol:13 data:[72 105 10 89 111 197 190 196 141 10 48 49 50 51 52 53 54 55 56 57 48 49 50 51 52 53 54 55 0 51 52 53 54 55] lines:{start:2 end:10 lengths:[3 7 18 0 0 0 0 0 0 0]}}
Hi
Yožč_|_
012345678901234567
DOWN {start:9 end:15 wantsCol:13 data:[72 105 10 89 111 197 190 196 141 89 111 197 190 196 141 10 48 49 50 51 52 53 54 55 56 57 48 49 50 51 52 53 54 55] lines:{start:1 end:9 lengths:[3 7 18 0 0 0 0 0 7 18]}}
Hi
Yožč
0123456789012_|_34567
DOWN {start:23 end:29 wantsCol:13 data:[72 105 10 89 111 197 190 196 141 10 48 49 50 51 52 53 54 55 56 57 48 49 50 55 56 57 48 49 50 51 52 53 54 55] lines:{start:2 end:10 lengths:[3 7 18 0 0 0 0 0 7 18]}}
Hi
Yožč
0123456789012_|_34567
DOWN {start:23 end:29 wantsCol:13 data:[72 105 10 89 111 197 190 196 141 10 48 49 50 51 52 53 54 55 56 57 48 49 50 55 56 57 48 49 50 51 52 53 54 55] lines:{start:2 end:10 lengths:[3 7 18 0 0 0 0 0 7 18]}}
Hi
Yožč
0123456789012
_|_34567
NEWL {start:24 end:29 wantsC

In [4]:
%%

gb := New()
fmt.Println(gb)

_|_
